In [27]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from keras import backend as K
from keras.layers import Layer,InputSpec
import keras.layers as kl
from glob import glob
from sklearn.metrics import roc_curve, auc
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from sklearn.metrics import roc_auc_score
from tensorflow.keras import callbacks 
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from  matplotlib import pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate,Dense, Conv2D, MaxPooling2D, Flatten,Input,Activation,add,AveragePooling2D,GlobalAveragePooling2D,BatchNormalization,Dropout
%matplotlib inline
import shutil
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from tensorflow.python.platform import build_info as tf_build_info
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
image_dir        = "D:\\Dataset\\HAM10000_Images_All"
metadata_csv     = "D:\\Dataset\\HAM10000_MetaData_CSV\\HAM10000_metadata.csv"
path_splitter    = "\\"
images_type      = ".jpg"

re_organized_dir = "D:\\Dataset\\re_organized"

In [3]:
data_pd = pd.read_csv(metadata_csv)
data_pd.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern


In [4]:
train_dir = os.path.join(image_dir, 'train_dir\\')
test_dir = os.path.join(image_dir, 'test_dir\\')

df_count = data_pd.groupby('lesion_id').count()
df_count.head()

df_count = df_count[df_count['dx'] == 1]

In [5]:
df_count.reset_index(inplace=True)

unique = set(df_count['lesion_id'])
def duplicates(x):
    if x in unique:
        return 'no' 
    else:
        return 'duplicates'
data_pd['is_duplicate'] = data_pd['lesion_id'].apply(duplicates)
data_pd.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,is_duplicate
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,duplicates
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,duplicates
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,duplicates
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,duplicates
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,duplicates


In [6]:
df_count = data_pd[data_pd['is_duplicate'] == 'no']

In [7]:
train, test_df = train_test_split(df_count, test_size=0.15, stratify=df_count['dx'])

test_data = set(test_df['image_id'])
def identify_trainOrtest(x):
    if str(x) in test_data:
        return 'test'
    else:
        return 'train'
    
#creating train_df
data_pd['train_test_split'] = data_pd['image_id'].apply(identify_trainOrtest)
train_df = data_pd[data_pd['train_test_split'] == 'train']
train_df.head()

# Image id of train and test images
train_list = list(train_df['image_id'])
test_list = list(test_df['image_id'])

# Set the image_id as the index in data_pd
data_pd.set_index('image_id', inplace=True)

In [8]:
train_dir = os.path.join(image_dir, 'train_dir\\')
test_dir = os.path.join(image_dir, 'test_dir\\')

In [9]:
batch_size = 4

In [10]:
train_path  = train_dir
test_path=test_dir

In [11]:
datagen=ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_resnet_v2.preprocess_input)

image_size = 224
print("\nTrain Batches: ")
train_batches = datagen.flow_from_directory(directory=train_path,
                                            target_size=(image_size,image_size),
                                            batch_size=batch_size,
                                            shuffle=True)

print("\nTest Batches: ")
test_batches =datagen.flow_from_directory(test_path,
                                           target_size=(image_size,image_size),
                                           batch_size=batch_size,
                                           shuffle=False)


Train Batches: 
Found 52411 images belonging to 7 classes.

Test Batches: 
Found 828 images belonging to 7 classes.


In [12]:
MainInput=Input(shape=(224, 224, 3))

In [14]:
def convlayer1(input_value):
    conv1=Conv2D(filters=64, kernel_size=(3,3), strides=(2,2),activation="relu",padding="same")(input_value)
    conv1=BatchNormalization()(conv1)
    pool1=MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)
    return pool1

In [17]:
def convlayer2(input_value):
    conv2=Conv2D(filters=64, kernel_size=(3,3),activation="relu",padding="same")(input_value)
    conv2=BatchNormalization()(conv2)
    conv2=Conv2D(filters=64, kernel_size=(3,3),activation="relu",padding="same")(conv2)
    conv2=BatchNormalization()(conv2)

    resnet=add([input_value,conv2])
    resnet=Activation("relu")(resnet)
    return resnet

In [18]:
def convlayer3(input_value):
    conv3=Conv2D(filters=128, kernel_size=(3,3),activation="relu",padding="same")(input_value)
    conv3=BatchNormalization()(conv3)
    conv3=Conv2D(filters=128, kernel_size=(3,3),activation="relu",padding="same")(conv3)
    conv3=BatchNormalization()(conv3)


    skip=Conv2D(filters=128, kernel_size=(3,3),activation="relu",padding="same")(input_value)
    skip=BatchNormalization()(skip)

    resnet=add([skip,conv3])
    resnet=Activation("relu")(resnet)
    return resnet

In [19]:
def convlayer4(input_value):
    conv4=Conv2D(filters=256, kernel_size=(3,3),activation="relu",padding="same")(input_value)
    conv4=BatchNormalization()(conv4)
    conv4=Conv2D(filters=256, kernel_size=(3,3),activation="relu",padding="same")(conv4)
    conv4=BatchNormalization()(conv4)


    skip=Conv2D(filters=256, kernel_size=(3,3),activation="relu",padding="same")(input_value)
    skip=BatchNormalization()(skip)

    resnet=add([skip,conv4])
    resnet=Activation("relu")(resnet)
    return resnet

In [20]:
def convlayer5(input_value):
    conv5=Conv2D(filters=512, kernel_size=(3,3),activation="relu",padding="same")(input_value)
    conv5=BatchNormalization()(conv5)
    conv5=Conv2D(filters=512, kernel_size=(3,3),activation="relu",padding="same")(conv5)
    conv5=BatchNormalization()(conv5)


    skip=Conv2D(filters=512, kernel_size=(3,3),activation="relu",padding="same")(input_value)
    skip=BatchNormalization()(skip)

    resnet=add([skip,conv5])
    resnet=Activation("relu")(resnet)
    return resnet

In [21]:
block1=convlayer1(MainInput)

In [22]:
block2=convlayer2(block1)
for i in range(0,2):
    block2=convlayer2(block2)

In [23]:
maxpool=MaxPooling2D(pool_size=(2,2), padding='same')(block2)
block3=convlayer3(maxpool)

for i in range(0,3):
    block3=convlayer3(block3)

In [24]:
block4=convlayer4(block3)
for i in range(0,5):
    block4=convlayer4(block4)

In [25]:
maxpool=MaxPooling2D(pool_size=(2,2), padding='same')(block4)
block5=convlayer5(maxpool)
for i in range(0,2):
    block5=convlayer5(block5)

In [28]:
output = GlobalAveragePooling2D()(block5)
output = Dense(7, activation='softmax')(output)
model = Model(inputs=MainInput, outputs=output)

In [29]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 112, 112, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

 batch_normalization_10 (BatchN  (None, 28, 28, 128)  512        ['conv2d_10[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_12 (Conv2D)             (None, 28, 28, 128)  147584      ['activation_3[0][0]']           
                                                                                                  
 conv2d_11 (Conv2D)             (None, 28, 28, 128)  147584      ['batch_normalization_10[0][0]'] 
                                                                                                  
 batch_normalization_12 (BatchN  (None, 28, 28, 128)  512        ['conv2d_12[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_nor

 conv2d_24 (Conv2D)             (None, 28, 28, 256)  590080      ['activation_7[0][0]']           
                                                                                                  
 conv2d_23 (Conv2D)             (None, 28, 28, 256)  590080      ['batch_normalization_22[0][0]'] 
                                                                                                  
 batch_normalization_24 (BatchN  (None, 28, 28, 256)  1024       ['conv2d_24[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_23 (BatchN  (None, 28, 28, 256)  1024       ['conv2d_23[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_8 (Ad

                                                                                                  
 batch_normalization_36 (BatchN  (None, 28, 28, 256)  1024       ['conv2d_36[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_35 (BatchN  (None, 28, 28, 256)  1024       ['conv2d_35[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_12 (Add)                   (None, 28, 28, 256)  0           ['batch_normalization_36[0][0]', 
                                                                  'batch_normalization_35[0][0]'] 
                                                                                                  
 activatio

In [30]:
opt1=tf.keras.optimizers.Adam(learning_rate=0.01,epsilon=0.1)
model.compile(optimizer=opt1,
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [31]:
class_weights = {   
                    0: 1.0,  # akiec
                    1: 1.0,  # bcc
                    2: 1.0,  # bkl
                    3: 1.0,  # df
                    4: 5.0,  # mel
                    5: 1.0,  # nv
                    6: 1.0,  # vasc
                }


checkpoint=  ModelCheckpoint(filepath = 'ResNet34.hdf5',monitor='val_accuracy',save_best_only=True,save_weights_only=True)

In [32]:
Earlystop = EarlyStopping(monitor='val_loss', mode='min',patience=65, min_delta=0.001)
history = model.fit(train_batches,
                    steps_per_epoch=(len(train_df)/10),
                    epochs=300,
                    verbose=2,
                    validation_data=test_batches,validation_steps=len(test_df)/batch_size,callbacks=[checkpoint,Earlystop],class_weight=class_weights)

Epoch 1/300


KeyboardInterrupt: 